In [5]:
import os

from langchain.chains import RetrievalQA
from PyPDF2 import PdfReader
from langchain.callbacks.base import BaseCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Neo4jVector
from langchain_community.chat_models import ChatOllama
from langchain_core.documents import Document
import os
import json
from semantic_text_splitter import TextSplitter
# load api key lib
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.embeddings import OllamaEmbeddings
load_dotenv(".env")

url = "neo4j://localhost:7687"
# url = "neo4j://localhost:7474"
username ="neo4j"
password = "password"

os.environ["NEO4J_URI"] = url #"bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] =username # "neo4j"
os.environ["NEO4J_PASSWORD"] = password #"password"


url = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
ollama_base_url = os.getenv("OLLAMA_BASE_URL")
embedding_model_name = os.getenv("EMBEDDING_MODEL")
llm_name = "medllama2"#os.getenv("LLM")
# Remapping for Langchain Neo4j integration
os.environ["NEO4J_URL"] = url

# logger = get_logger(__name__)


# embeddings, dimension = load_embedding_model(
#     embedding_model_name, config={"ollama_base_url": ollama_base_url}, logger=logger
# )


# class StreamHandler(BaseCallbackHandler):
#     def __init__(self, container, initial_text=""):
#         self.container = container
#         self.text = initial_text

#     def on_llm_new_token(self, token: str, **kwargs) -> None:
#         self.text += token
#         self.container.markdown(self.text)


# llm = load_llm(llm_name, logger=logger, config={"ollama_base_url": ollama_base_url})


embeddings = OllamaEmbeddings(
             model="medllama2"
        )

max_characters = 100
# Optionally can also have the splitter not trim whitespace for you
splitter = TextSplitter()

# Document(page_content=sect['text'])

folder_path = "/workspaces/thyroid_knowledge_graph/preprocessed"
file_paths = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)

res=[]
for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        sections= data['sections']
        for sect in sections:
            # res.append(sect['text'])
            for chunk in splitter.chunks(sect['text'], max_characters):

                res.append(chunk)

llm_med = ChatOllama(model="medllama2")




In [6]:
chunks = res[0:30]
chunks


['Thyroid cancer is divided into differentiated thyroid cancer (DTC) and undifferentiated thyroid',
 'cancer.',
 'In contrast to the undifferentiated thyroid cancer, the vast majority of the DTC accumulates iodine',
 'in the tumour cells.',
 'Utilizing this characteristic, DTC patients can undergo radioiodine therapy (RIT).',
 'RIT of DTC patients provides a very effective therapy with only minimal side effects using standard',
 'activities, which is established for more than half a century.',
 'Patients are treated according to their tumour stage.',
 'Low-risk patients usually undergo a single RIT with activities up to 4 GBq 131 I.',
 'High-risk patients undergo at least one or several RIT with higher activities [1,2].',
 'While performing RIT with high activities of 131 I, the organs at risk need specific attention.',
 'These are the bone marrow and the lung, especially in case of multiple metastases.',
 'The amount of 131 I activity and the number of RIT depend on the results of the

In [7]:
# Store the chunks part in db (vector)
vectorstore = Neo4jVector.from_texts(
    chunks,
    url=url,
    username=username,
    password=password,
    embedding=embeddings,
    index_name="pdf_bot",
    node_label="PdfBotChunk",
    pre_delete_collection=True,  # Delete existing PDF data
)


In [9]:
vector_qa = RetrievalQA.from_chain_type(
    llm=llm_med,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)


"RIT stands for Radioimmunotherapy and it's a treatment that combines radiation with immune system to fight cancer cells. It can be used in cases where the patient has recurrences or persistent disease after previous treatments, and they have already undergone radioiodine therapy. However, I am not sure what specific situation you are referring to and it's always best to consult a medical professional for more information.\n\nThe user is asking about Radioimmunotherapy (RIT), which is a treatment that combines radiation with the immune system to fight cancer cells. RIT is used in cases where patients have recurrences or persistent disease after previous treatments, and they have already undergone radioiodine therapy."

In [10]:
vector_qa.run(
    "what is RIT"
)

"While I am familiar with the topic of RIT for thyroid cancer, I'm not sure about the specific case you are referring to. Please provide more context so that I can answer your question accurately."